In [ ]:
#load dataset
import pandas as pd
import matplotlib.pyplot as pl

DATASET_COLUMNS = ["target", "id", "time", "query", "to", "posts"]
data=pd.read_csv('data.csv', names=DATASET_COLUMNS , encoding='ISO-8859-1')
data.head()

In [ ]:
#Dataset Analysis
print(data["query"].unique())
data['target'].unique()
data.dtypes
data['target'] = data['target'].replace(4,1)
data['target'].unique()
#data['target'].nunique()

In [ ]:
# grouping data according to target
target_grouped =  data.groupby('target')['target'].count
target_grouped


In [ ]:
#splitting data to lists
target_list = list(data['target'])
text_list = list(data['posts'])
target_list
text_list

In [ ]:
#visualizing data distribution according to target
import seaborn as sns
sns.countplot(x='target', data=data)

In [ ]:
# spliting positive and negative data
data_pos=data[data['target']==1]
data_pos
data_neg=data[data['target']==0]

In [ ]:
#reducing data size to 25%
data_pos=data_pos.iloc[:int(200000)]
data_neg=data_neg.iloc[:int(200000)]

In [ ]:
# removing punctuations except @
import string
english_punctuations = string.punctuation
english_punctuations = english_punctuations.replace("@", "")
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
data['posts']= data['posts'].apply(lambda x: cleaning_punctuations(x))
data['posts'].head()


In [ ]:
# removing username
import re
pattern = '@\S+'
def remove_pattern(text, pattern):
    r = re.findall(pattern,text)
    for i in r:
        text = re.sub(i,"",text)
    return text

data['posts'] = data['posts'].apply(lambda x: remove_pattern(x, pattern))
data['posts']

In [ ]:
# removing url
import re
pattern_url = 'http\S+'
def remove_pattern(text, pattern):
    r = re.findall(pattern,text)
    for i in r:
        text = re.sub(i,"",text)
    return text

data['posts'] = data['posts'].apply(lambda x: remove_pattern(x, pattern_url))
data['posts']

In [ ]:
#removing numbers
pattern_num = '[0-9]'
data['posts'] = data['posts'].apply(lambda x: remove_pattern(x, pattern_num))
data['posts']

In [18]:
# tokenization

data_posts_tokenized = data['posts'].apply(lambda x: x.split())
data_posts_tokenized

# https://towardsdatascience.com/5-simple-ways-to-tokenize-text-in-python-92c6804edfc4 @jatin 

0          [Awww, thats, a, bummer, You, shoulda, got, Da...
1          [is, upset, that, he, cant, update, his, Faceb...
2          [I, dived, many, times, for, the, ball, Manage...
3          [my, whole, body, feels, itchy, and, like, its...
4          [no, its, not, behaving, at, all, im, mad, why...
                                 ...                        
1599995    [Just, woke, up, Having, no, school, is, the, ...
1599996    [TheWDBcom, Very, cool, to, hear, old, Walt, i...
1599997    [Are, you, ready, for, your, MoJo, Makeover, A...
1599998    [Happy, th, Birthday, to, my, boo, of, alll, t...
1599999                              [happy, charitytuesday]
Name: posts, Length: 1600000, dtype: object

In [ ]:
# remove stop words, stemming we can skip, lemmatization

In [ ]:
# display(data.to_string())
# data['posts'][3]

In [ ]:
# # remove punctuations
# import string
# import re
# punctuation = string.punctuation
# punctuation = punctuation.replace("@", "")
# punctuation_list = list(punctuation)

# """
# def remove_punctuation(text) :
#     for i in punctuation_list :
#         if (text.find(i)!=-1) :
#             text.replace(i, " ")
#     return text
# """

# def remove_punct(text):
#     new_words = []
#     for word in text:
#         w = re.sub(r'[^\w\s\@]','',word) #remove everything except words and space
#         w = re.sub(r'_','',w) #how to remove underscore as well
#         new_words.append(w)
#     return ''.join(str(e) for e in new_words)
    

# data['posts'] = data['posts'].apply(lambda x : remove_punct(x))
# data['posts']
# new_data = data['posts']

# #data['posts'] = data['posts'].str.replace("[^@a-zA-Z#]", " ")
# #data['posts']


In [ ]:
#print(data['posts'][0])

#for i in range(80000,800000) :
#    data['posts'][i]=remove_pattern(data['posts'][i], pattern)